In [8]:
#TASK 1
from queue import PriorityQueue

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start node to current node
        self.h = 0  # Heuristic estimate to nearest goal
        self.f = 0  # Total cost (for Best-First Search)

    def __lt__(self, other):
        return self.f < other.f

# Manhattan Distance Heuristic
def heuristic(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def best_first_search(maze, start, goals):
    all_goals = set(goals)
    visited_goals = set()

    current_start = start
    final_path = []

    while visited_goals != all_goals:
        nearest_goal = min(all_goals - visited_goals, key=lambda g: heuristic(current_start, g))
        path = find_path(maze, current_start, nearest_goal)
        if not path:
            return None

        final_path.extend(path[:-1])  # Avoid duplicate goal positions
        visited_goals.add(nearest_goal)
        current_start = nearest_goal

    final_path.append(current_start)
    return final_path

def find_path(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position

        if current_pos == end:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        visited.add(current_pos)
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited:
                new_node = Node(new_pos, current_node)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h  # Best-First Search: f(n) = h(n)
                frontier.put(new_node)
                visited.add(new_pos)

    return None

maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goals = [(2, 3), (4, 4), (1, 1)]
path = best_first_search(maze, start, goals)

if path:
    print("Path covering all goals:", path)
else:
    print("No path found")


Path covering all goals: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]


In [2]:
#TASK 2
import random
import time

def a_star(graph, start, goal, heuristic):
    frontier = [(start, 0 + heuristic[start])]
    visited = set()
    g_costs = {start: 0}
    came_from = {start: None}

    while frontier:
        # Sort frontier by f(n) = g(n) + h(n)
        frontier.sort(key=lambda x: x[1])
        current_node, current_f = frontier.pop(0)

        if current_node in visited:
            continue

        print(current_node, end=" ")
        visited.add(current_node)

        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = came_from[current_node]
            path.reverse()
            print(f"\nGoal found with A*. Path: {path}")
            return

        for neighbor in list(graph[current_node].keys()):
            graph[current_node][neighbor] = random.randint(1, 10)
            cost = graph[current_node][neighbor]

            new_g_cost = g_costs[current_node] + cost
            f_cost = new_g_cost + heuristic[neighbor]

            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                came_from[neighbor] = current_node
                frontier.append((neighbor, f_cost))

        time.sleep(1)  # Simulate real-time updates

    print("\nGoal not found")

graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 3},
    'D': {'B': 2, 'G': 6},
    'E': {'B': 5, 'G': 2},
    'F': {'C': 3, 'G': 4},
    'G': {'D': 6, 'E': 2, 'F': 4}
}

heuristic = {
    'A': 7, 'B': 6, 'C': 2, 'D': 4, 'E': 3, 'F': 1, 'G': 0
}

print("\nFollowing is the A* Search with dynamic edge costs:")
a_star(graph, 'A', 'G', heuristic)



Following is the A* Search with dynamic edge costs:
A C F G 
Goal found with A*. Path: ['A', 'C', 'F', 'G']


In [7]:
#TASK 3
import random
import time

def greedy_bfs(city_map, start, deliveries, heuristic, time_windows):
    frontier = [(start, heuristic[start], 0)]  # (node, heuristic, time_elapsed)
    visited = set()
    came_from = {start: None}
    delivery_order = []

    while frontier:
        # Sort frontier by heuristic (priority on closer deliveries)
        frontier.sort(key=lambda x: (x[1], time_windows.get(x[0], float('inf'))))
        current_node, _, current_time = frontier.pop(0)

        if current_node in visited:
            continue

        print(f"Delivering to: {current_node}")
        visited.add(current_node)

        if current_node in deliveries:
            delivery_order.append(current_node)
            deliveries.remove(current_node)

            if not deliveries:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                print(f"\nOptimized delivery route: {path}")
                return

        for neighbor in city_map.get(current_node, {}):
            traffic_factor = random.uniform(0.8, 2.0)
            city_map[current_node][neighbor] =int(city_map[current_node][neighbor] * traffic_factor)
            cost = city_map[current_node][neighbor]

            if neighbor not in visited:
                came_from[neighbor] = current_node
                frontier.append((neighbor, heuristic[neighbor], current_time + cost))

        time.sleep(1)

    print("\nNo feasible delivery route found.")

city_map = {
    'A': {'B': 5, 'C': 10},
    'B': {'A': 5, 'D': 15, 'E': 20},
    'C': {'A': 10, 'F': 25},
    'D': {'B': 15, 'G': 30},
    'E': {'B': 20, 'G': 10},
    'F': {'C': 25, 'G': 15},
    'G': {'D': 30, 'E': 10, 'F': 15}
}

heuristic = {
    'A': 40, 'B': 30, 'C': 20, 'D': 15, 'E': 10, 'F': 5, 'G': 0
}
deliveries = {'E', 'F', 'G'}
time_windows = {'E': 30, 'F': 50, 'G': 60}

print("\nGreedy Best-First Search considering time windows:")
greedy_bfs(city_map, 'A', deliveries, heuristic, time_windows)


Greedy Best-First Search considering time windows:
Delivering to: A
Delivering to: C
Delivering to: F
Delivering to: G
Delivering to: E

Optimized delivery route: ['A', 'C', 'F', 'G', 'E']


In [6]:
#TASK 4
import random
import time

def a_star(city_map, start, goal, heuristic):
    frontier = [(start, 0 + heuristic[start])]
    visited = set()
    g_costs = {start: 0}
    came_from = {start: None}

    while frontier:
        # Sort frontier by f(n) = g(n) + h(n)
        frontier.sort(key=lambda x: x[1])
        current_node, current_f = frontier.pop(0)

        if current_node in visited:
            continue

        print(f"Visiting: {current_node}")
        visited.add(current_node)

        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = came_from[current_node]
            path.reverse()
            print(f"\nFastest route found: {path}")
            return

        for neighbor in list(city_map[current_node].keys()):
            traffic_factor = random.uniform(0.8, 2.0)  # Random congestion multiplier
            city_map[current_node][neighbor] =  int(city_map[current_node][neighbor] * traffic_factor)
            cost = city_map[current_node][neighbor]

            new_g_cost = g_costs[current_node] + cost
            f_cost = new_g_cost + heuristic[neighbor]

            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                came_from[neighbor] = current_node
                frontier.append((neighbor, f_cost))

        time.sleep(1)

    print("\nNo route found due to heavy traffic.")

city_map = {
    'A': {'B': 5, 'C': 10},
    'B': {'A': 5, 'D': 15, 'E': 20},
    'C': {'A': 10, 'F': 25},
    'D': {'B': 15, 'G': 30},
    'E': {'B': 20, 'G': 10},
    'F': {'C': 25, 'G': 15},
    'G': {'D': 30, 'E': 10, 'F': 15}
}

heuristic = {
    'A': 40, 'B': 30, 'C': 20, 'D': 15, 'E': 10, 'F': 5, 'G': 0
}

print("\nA* Search considering real-time traffic updates:")
a_star(city_map, 'A', 'G', heuristic)


A* Search considering real-time traffic updates:
Visiting: A
Visiting: B
Visiting: E
Visiting: C
Visiting: G

Fastest route found: ['A', 'B', 'E', 'G']
